In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/churn-modelling/Churn_Modelling.csv")
df.head()

In [ ]:
missing_cols = [col for col in df.columns if df[col].isnull().any()]
missing_cols

In [ ]:
# Categorical columns
df_categorical = df.select_dtypes('object')

In [ ]:
from sklearn.preprocessing import LabelEncoder
df_c = df.copy()
label_encoder = LabelEncoder()
for col in df_categorical.columns:
    df_c[col] = label_encoder.fit_transform(df[col])

In [ ]:
y = df_c[['Exited']]
df_c.drop('Exited', axis=1, inplace=True)
X = df_c

In [ ]:
from sklearn.model_selection import train_test_split
X_train_c, X_test_c, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Standardization
standard_scaler = MinMaxScaler(feature_range=(0, 1))
X_train = pd.DataFrame(standard_scaler.fit_transform(X_train_c))
X_test = pd.DataFrame(standard_scaler.fit_transform(X_test_c))
print(X_train.head())

In [ ]:
X_train.shape[1]

In [ ]:
import keras as keras
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[X_train.shape[1]]))
model.add(keras.layers.Dense(100, kernel_initializer="he_normal", activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(100, kernel_initializer="he_normal", activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(100, kernel_initializer="he_normal", activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(100, kernel_initializer="he_normal", activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(2, activation="softmax"))
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history=model.fit(X_train, y_train, epochs=100)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
p = np.argmax(model.predict(X_test), axis=1)
p

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

print(classification_report(y_test, p))

# Confusion matrix for testing dataset
labels = [0,1]
cm = confusion_matrix(y_test, p, labels)
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
